In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
from dotenv import load_dotenv
import os


load_dotenv(override=True)
pghost = os.getenv('PGHOST')
pguser = os.getenv('PGUSER')
pgpassword = os.getenv('PGPASSWORD')
pgdatabase = os.getenv('PGDATABASE')
pgport = os.getenv('PGPORT')

engine=create_engine(f'postgresql://{pguser}:{pgpassword}@{pghost}:{pgport}/{pgdatabase}')

with engine.connect() as con:
    with open("query.sql") as file:
        query = text(file.read())
        con.execute(query)

#### 1. Simple SELECT
Write a query to retrieve the names of all employees and their corresponding departments.

In [3]:
query_1 = "SELECT name, department FROM employees;"
result_question_1 = pd.read_sql_query(query_1, engine)
result_question_1

,name,department
0,Alice,Engineering
1,Bob,Marketing
2,Charlie,Engineering
3,David,Engineering
4,Eve,HR
5,Frank,Marketing
6,Grace,Finance
7,Henry,Finance
8,Ivy,HR
9,John,Engineering


#### 2. Filtering with WHERE

Write a query to retrieve the names of all employees who work in the ‘Engineering’ department.

In [4]:
query_2 = "SELECT name, department FROM employees WHERE department = 'Engineering';"
result_question_2 = pd.read_sql(query_2, engine)
result_question_2

,name,department
0,Alice,Engineering
1,Charlie,Engineering
2,David,Engineering
3,John,Engineering


#### 3. Ordering Results

Write a query to retrieve the names and salaries of all employees, ordered by salary in descending order.

In [19]:
query_3 = "SELECT name, salary FROM employees ORDER BY salary DESC;"
result_question_3 = pd.read_sql(query_3, engine)
result_question_3

,name,salary
0,Henry,85000.0
1,Grace,70000.0
2,David,65000.0
3,John,62000.0
4,Alice,60000.0
5,Charlie,55000.0
6,Frank,51000.0
7,Bob,50000.0
8,Eve,48000.0
9,Ivy,47000.0


#### 4. Aggregate Function

Write a query to count the number of employees in each department.

In [20]:
query_4 = "SELECT department, COUNT(id) FROM employees GROUP BY department;"
result_question_4 = pd.read_sql(query_4, engine)
result_question_4


,department,count
0,Marketing,2
1,Finance,2
2,Engineering,4
3,HR,2


#### 5. Find the total hours worked per project

Write a query that retrieves the project names and the total hours worked by all employees on each project.



In [21]:
query_5 = """SELECT projects.project_name, SUM(assignments.hours_worked)
            FROM projects 
            JOIN assignments ON assignments.project_id = projects.project_id
            GROUP BY projects.project_id;
            """
result_question_5 = pd.read_sql(query_5, engine)
result_question_5

,project_name,sum
0,Project C,250
1,Project D,170
2,Project E,190
3,Project A,270
4,Project B,220


#### 6. Projects with more than 200 total hours worked

Write a query that retrieves the project names where the total hours worked by all employees exceed 200 hours.

In [22]:
query_6 = """SELECT projects.project_name, SUM(assignments.hours_worked) AS total_hours_worked
            FROM projects 
            JOIN assignments ON assignments.project_id = projects.project_id
            GROUP BY projects.project_id
            HAVING SUM(assignments.hours_worked) > 200;
            """
result_question_6 = pd.read_sql(query_6, engine)
result_question_6

,project_name,total_hours_worked
0,Project C,250
1,Project A,270
2,Project B,220


#### 7. Classify employees based on their hours worked

Write a query that retrieves each employee's ID, their hours worked, and a classification based on hours worked:

- "Overworked" if they worked more than 120 hours
- "Normal" if they worked between 80 and 120 hours
- "Underworked" if they worked less than 80 hours

In [23]:
query_7 = """SELECT employee_id, hours_worked,
            CASE 
                WHEN hours_worked BETWEEN 80 AND 120 THEN 'Normal'
                WHEN hours_worked > 120 THEN 'Overworked'
                ELSE 'Underworked' 
                END AS classification
            FROM assignments;
            """
result_question_7 = pd.read_sql(query_7, engine)
result_question_7

,employee_id,hours_worked,classification
0,1,120,Normal
1,2,100,Normal
2,3,150,Overworked
3,4,140,Overworked
4,5,80,Normal
5,6,120,Normal
6,7,110,Normal
7,8,90,Normal
8,9,60,Underworked
9,10,130,Overworked


#### 8. JSON Extraction 

Extract the primary skill from the info JSON column, but only for employees in the “Engineering” department who have been managed by “Alice” and earn more than 60,000.

In [24]:
query_8 = """SELECT name, info->'skills'->'primary' AS primary_skill 
            FROM employees
            WHERE department = 'Engineering' AND info->>'manager' = 'Alice' AND salary > 60000;
            """
result_question_8 = pd.read_sql(query_8, engine)
result_question_8

,name,primary_skill
0,David,Go
1,John,C++


#### 9. Classify projects based on total hours worked

Write a query that retrieves the project names and classifies them based on the total hours worked on each project:

- "High Workload" if the total hours worked exceed 500 hours
- "Medium Workload" if the hours worked are between 200 and 500 hours
- "Low Workload" if the hours worked are less than 200 hours

Display only those projects that have more than 100 hours worked.

In [25]:
query_9 = """SELECT projects.project_name, SUM(assignments.hours_worked) AS total_hours_worked,
        CASE 
            WHEN SUM(assignments.hours_worked) < 200 THEN 'Low Workload'
            WHEN SUM(assignments.hours_worked) BETWEEN 200 AND 500 THEN 'Medium Workload'
            ELSE 'High Workload'
        END AS classification_proj
        FROM projects 
        JOIN assignments ON assignments.project_id = projects.project_id
        GROUP BY projects.project_id;
        """
classify_projects = pd.read_sql(query_9, engine)
classify_projects

,project_name,total_hours_worked,classification_proj
0,Project C,250,Medium Workload
1,Project D,170,Low Workload
2,Project E,190,Low Workload
3,Project A,270,Medium Workload
4,Project B,220,Medium Workload


#### 10. Using CTE 

Find the average salary of employees hired before January 1, 2020, separated by department, but only include departments with a budget greater than 300,000.

In [26]:
query_10 = """WITH FilteredEmployees AS (
    SELECT employees.department, employees.salary
    FROM employees
    JOIN departments ON employees.department = departments.department_name
    WHERE employees.hire_date < '2020-01-01' AND departments.budget > 300000
    )
    SELECT departments.department_name,
        AVG(FilteredEmployees.salary) AS average_salary
    FROM FilteredEmployees
    JOIN departments ON FilteredEmployees.department = departments.department_name
    GROUP BY departments.department_name;
    """
result_question_10 = pd.read_sql(query_10, engine)
result_question_10

,department_name,average_salary
0,Engineering,55000.0
1,Finance,77500.0


#### 11. Handling Timestamps

Display the total number of hours worked by employees grouped by month, but only for projects that have a budget greater than 150,000 and started in 2023.

In [27]:
query_11 = """SELECT 
            DATE_TRUNC('month', projects.start_date) AS month,
            SUM(assignments.hours_worked) AS total_hours_worked
        FROM projects
        JOIN assignments ON projects.project_id = assignments.project_id
        JOIN employees ON assignments.employee_id = employees.id
        JOIN departments ON employees.department = departments.department_name
        WHERE 
            projects.start_date >= '2023-01-01'
            AND projects.start_date < '2024-01-01'
            AND departments.budget > 150000
        GROUP BY DATE_TRUNC('month', projects.start_date)
        ORDER BY month;
    """
result_question_11 = pd.read_sql(query_11, engine)
result_question_11

,month,total_hours_worked
0,2023-01-01,270
1,2023-02-01,220
2,2023-03-01,250
3,2023-04-01,170
4,2023-05-01,190


#### 12. Window Functions - Ranking Employees by Salary with Department Filters

Rank employees by salary within their department, but only include departments with a budget greater than 400,000 and employees hired after January 1, 2019.

In [28]:
query_12 = """WITH FilteredEmployees AS (
    SELECT 
        employees.id,
        employees.name,
        employees.department,
        employees.salary,
        departments.department_name
    FROM employees
    JOIN departments ON employees.department = departments.department_name
    WHERE 
        departments.budget > 400000
        AND employees.hire_date > '2019-01-01'
    )
    SELECT 
        name,
        salary,
        department_name,
        RANK() OVER (PARTITION BY department ORDER BY salary DESC) AS salary_rank
    FROM FilteredEmployees
    ORDER BY department_name, salary_rank;
    """
result_question_12 = pd.read_sql(query_12, engine)
result_question_12

,name,salary,department_name,salary_rank
0,David,65000.0,Engineering,1
1,John,62000.0,Engineering,2
2,Alice,60000.0,Engineering,3


#### 14. Subqueries with Multiple Filters

Find the project(s) that has the highest total hours worked by employees, but only include employees in departments with a budget greater than 300,000 and projects that started in 2023.

In [29]:
query_14 = """SELECT 
    projects.project_id, 
    projects.project_name, 
    total_hours.total_hours_worked
FROM projects 
JOIN (
    SELECT assignments.project_id,
        SUM(assignments.hours_worked) AS total_hours_worked
    FROM assignments 
    JOIN employees ON assignments.employee_id = employees.id
    JOIN departments ON employees.department = departments.department_name
    WHERE departments.budget > 300000
    GROUP BY assignments.project_id
) AS total_hours ON projects.project_id = total_hours.project_id
WHERE 
    projects.start_date >= '2023-01-01'
    AND projects.start_date < '2024-01-01'
    AND total_hours.total_hours_worked = (
        SELECT MAX(sub.total_hours_worked)
        FROM (
            SELECT 
                assignments.project_id,
                SUM(assignments.hours_worked) AS total_hours_worked
            FROM assignments
            JOIN employees ON assignments.employee_id = employees.id
            JOIN departments ON employees.department = departments.department_name
            WHERE departments.budget > 300000
            GROUP BY assignments.project_id) AS sub
    );
    """
result_question_14 = pd.read_sql(query_14, engine)
result_question_14

,project_id,project_name,total_hours_worked
0,101,Project A,270


#### 13. Handling Duplicates with Additional Filters


Write a query to remove duplicate employee records based on name and department, but only include those with “Engineering” or “Marketing” in their department and hired before 2021.

In [30]:
### Run this at the end, because this will delete rows in the database

query_13 = """WITH DuplicateEmployees AS (
SELECT DISTINCT 
    employees.id, 
    employees.name, 
    employees.department, 
    ROW_NUMBER() OVER (PARTITION BY employees.name, employees.department ORDER BY employees.id) AS row_num
FROM employees
JOIN departments ON employees.department = departments.department_name
WHERE 
    (departments.department_name = 'Engineering' OR departments.department_name = 'Marketing')
    AND employees.hire_date < '2021-01-01')
DELETE FROM employees
WHERE id IN (
    SELECT id
    FROM DuplicateEmployees
    WHERE row_num > 1
);
"""
with engine.connect() as con:
    result_question_13 = con.execute(text(query_13))
    result_question_13